In [1]:
import pandas as pd
import numpy as np
import math
import random
import scipy
import statsmodels.formula.api as smf
import sklearn.linear_model as lm

import keras

import matplotlib.pyplot as plt
%matplotlib inline

/Users/dannytb/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
#Global Variables 
train_test_split = .8 #fraction of data to use for training

!ls

Final Project.ipynb    HRDataset_v13.csv      winequality-red.csv
Final_Project_v2.ipynb ph_v1_days.csv


In [28]:
pdth = pd.read_csv("winequality-red.csv")
print(pdth.head())
print("Total Availible Keys: ", len(pdth.keys()))

repl_nan = lambda x: list(map(lambda xx: 0 if math.isnan(xx) else xx, x))

qualtities = list(map(lambda x: int(x), repl_nan(pdth['quality'])))

xs = []
for k in pdth.keys():
    if k in ["quality"]:
        continue
    xs.append(repl_nan(list(map(lambda x: float(x), pdth[k]))))
    
# Put the data into numpy arrays
xs=np.transpose(np.array(xs))
ys=np.array(qualtities)

#Xshape = (num_samples, features), Yshape = (num_samples,)
print("\nPre Split Shapes:")
print("X shape = {}, Y shape = {}".format(xs.shape, ys.shape))

#Create a training and testing split

#Zip the xs and ys together
xsys = list(zip(xs, ys))

#shuffle the combined list
random.shuffle(xsys)

#Unzip the shuffled list
xs, ys = zip(*xsys)

splt_ndx = int(len(xs)*train_test_split)

xs_tr = np.array(xs[0:splt_ndx]); xs_ts = np.array(xs[splt_ndx:])
ys_tr = np.array(ys[0:splt_ndx]); ys_ts = np.array(ys[splt_ndx:])

#Print the final shapes
print("\nFinal Shapes:")
print("Shapes: X train = {}, Y train = {}".format(xs_tr.shape, ys_tr.shape))
print("        X test  = {},  Y test  = {}".format(xs_ts.shape, ys_ts.shape))

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [31]:
model_lr = lm.LogisticRegression(solver='liblinear')
model_lr.fit(xs_tr, ys_tr)

get_acc_lr = lambda x, y: len(np.where(model_lr.predict(x)==y)[0])/len(y)

print("Training Accuracy = ", get_acc_lr(xs_tr, ys_tr))
print("Testing  Accuracy = ", get_acc_lr(xs_ts, ys_ts))

print(model_lr.predict(xs_tr)[0:100])
print(ys_tr[0:100])

Training Accuracy =  0.5957779515246286
Testing  Accuracy =  0.56875
[6 6 5 5 5 6 7 6 6 6 5 5 6 5 5 5 5 6 5 5 6 5 6 5 5 6 6 6 5 6 6 5 6 6 5 6 6
 5 5 6 6 6 5 5 5 6 6 6 6 6 6 5 6 5 6 6 6 5 6 5 5 6 5 5 5 6 6 5 6 6 6 6 6 6
 6 5 6 6 6 6 5 5 6 5 6 5 6 5 5 6 5 6 6 6 5 5 6 6 6 5]
[6 5 5 5 5 5 6 7 6 7 6 5 7 5 5 6 6 7 6 6 6 5 6 5 5 5 6 7 4 6 6 5 6 6 5 5 6
 5 6 6 7 5 5 5 5 6 6 6 7 7 6 6 7 5 7 6 6 5 5 5 6 6 5 5 5 8 6 5 6 6 6 5 6 6
 6 5 6 6 6 6 6 5 6 5 7 5 5 5 5 6 5 6 6 6 7 6 6 6 7 5]


In [44]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

hddn_units = 64
model_ml = Sequential([
    Dense(hddn_units, activation='relu'), 
    Dropout(.5),
    Dense(1, activation='sigmoid')
])
model_ml.compile(optimizer='adam', loss='mse')

model_ml.fit(xs_tr, ys_tr, epochs=10, verbose=0)

print(model_ml.predict(xs_tr)[0:5])
print(ys_tr[0:5])

def get_acc_ml(x, y):
    preds = model_ml.predict(x)[:, 0]
    print(preds[0:5])
    preds = np.array(list(map(lambda x: int(round(x)), preds)))
    print(preds[0:5])
    print(y[0:5])
    
    return len(np.where(preds==y)[0])/len(y)

print("Training Accuracy = ", get_acc_ml(xs_tr, ys_tr))
print("Testing  Accuracy = ", get_acc_ml(xs_ts, ys_ts))



[[6.0130415]
 [6.094811 ]
 [5.0582004]
 [4.876585 ]
 [5.22621  ]]
[6 5 5 5 5]
[6.0130415 6.094811  5.0582004 4.876585  5.22621  ]
[6 6 5 5 5]
[6 5 5 5 5]
Training Accuracy =  0.4331508991399531
[6.214538  6.5077252 5.150707  5.375953  5.8831043]
[6 7 5 5 6]
[7 6 7 6 7]
Testing  Accuracy =  0.434375
